In [16]:
from pandas import read_csv
from numpy import array, unique, log2, inf, append, where
from sklearn.model_selection import train_test_split
import random
from pprint import pprint

In [2]:
file_path = '../catalog1/cat1.csv'
df = read_csv(file_path)
df.head()
y = array(df['class'])
# df = df.rename(columns={"class": "label"})
if 'cat2.csv' in file_path:
    df.drop("Unnamed: 0.1", axis=1, inplace=True)
df.drop(["Unnamed: 0", "galex_objid", "sdss_objid", "class", "spectrometric_redshift", "pred"], axis=1, inplace=True)

In [3]:
X = array(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [4]:
def check_purity(y):
    if len(unique(y)) == 1:
        return True
    else:
        return False

In [13]:
def classify_data(y):
    unique_classes, counts_unique_classes = unique(y, return_counts=True)

    index = where(counts_unique_classes == max(counts_unique_classes))[0][0]
    classification = unique_classes[index]
    
    return classification

In [6]:
def get_potential_splits(X, y):
    
    potential_splits = {}
    #n_columns = len(X[0])
    for column_index in range(len(X[0])):  
        potential_splits[column_index] = []
        values = X[:, column_index]
        unique_values = unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

In [7]:
def split_data(X, y, split_column, split_value):
    
    no_of_columns = len(X[0]) + 1
    split_column_values = X[:, split_column]
    data_below = array([]).reshape(0, no_of_columns)
    data_above = array([]).reshape(0, no_of_columns)
#     print("len(X) in function split_data:", len(X))
    for index in range(len(X) - 1):
        temp = array(append(X[index], y[index])).reshape(1, no_of_columns)
        if split_column_values[index] <= split_value:
            data_below = append(data_below, temp, axis=0)
        else:
            data_above = append(data_above, temp, axis=0)
    
    return data_below, data_above

In [8]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -log2(probabilities))
     
    return entropy

In [9]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [10]:
def determine_best_split(X, y, potential_splits):    
    overall_entropy = inf
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(X, y, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [23]:
def decision_tree_algorithm(X, y, column_headers, max_depth, counter=0, min_samples=2):
    
#     # data preparations
#     if counter == 0:
#         global COLUMN_HEADERS
#         COLUMN_HEADERS = df.columns
#         data = df.values
#     else:
#         data = df   
        
    if (check_purity(y)) or (len(X) < min_samples) or (counter == max_depth):
        classification = classify_data(y)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(X, y)
        split_column, split_value = determine_best_split(X, y, potential_splits)
        data_below, data_above = split_data(X, y, split_column, split_value)
    
        feature_name = column_headers[split_column]
        question = "{} <= {}".format(feature_name, split_value)
        sub_tree = {question: []}
        
        print(sub_tree)
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        return sub_tree

In [24]:
tree = decision_tree_algorithm(X_train, y_train, column_headers=df.columns, max_depth=3)
pprint(tree)

{'g-z <= -0.040340900421199005': []}
1


In [34]:
X_train.shape, y_train.shape

((454, 33), (195, 33))

In [25]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [26]:
example = X_test[0]
print(example)
classify_example(example, tree)

NameError: name 'test' is not defined

In [23]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, axis=1, args=(tree,))
    df["classification_correct"] = df["classification"] == df["label"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [24]:
from sklearn.metrics import accuracy_score
accuracy = calculate_accuracy(test, tree)
accuracy

NameError: name 'test' is not defined

In [7]:
454 * 33

14982

In [20]:
type(tree)

numpy.int64